In [1]:
import os
import sys
import pandas as pd
import numpy as np
import time
import warnings
import pickle

from tqdm import tqdm

warnings.filterwarnings("ignore")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

sys.path.append("../../")

from utils import DATA_DIR  # noqa

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [4]:
def unpickle(file):
    with open(file, "rb") as fo:
        d = pickle.load(fo, encoding="bytes")
    return d

def load_cifar(filenames):
    train_images = []
    train_labels = []

    for file_name in filenames:
        unpickled_images = unpickle(file_name)
        images, labels = unpickled_images[b"data"], unpickled_images[b"labels"]
        images = np.reshape(images, (-1, 3, 32, 32))
        images = np.transpose(images, (0, 2, 3, 1))
        train_images.append(images)
        train_labels += labels

    return np.vstack(train_images), train_labels

In [5]:
print("Loading the training set")
train_files = [f"../../data/cifar-10/data_batch_{i}" for i in range(1, 6)]
train_images, int_train_labels = load_cifar(train_files)

print("Loading the testing set")
test_files = ["../../data/cifar-10/test_batch"]
test_images, int_test_labels = load_cifar(test_files)

print("Loading the labels")
label_names = unpickle("../../data/cifar-10/batches.meta")[b"label_names"]

train_labels = [str(label_names[_]) for _ in int_train_labels]
test_labels = [str(label_names[_]) for _ in int_test_labels]

Loading the training set
Loading the testing set
Loading the labels


In [6]:
import torchvision.models as models

vgg = models.vgg16(pretrained=True)
vgg.classifier[-1].out_features = 10

normalized_train_images = (train_images / 255 - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
normalized_test_images = (test_images / 255 - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

In [7]:
train_images_tensor = torch.Tensor(normalized_train_images.transpose(0, 3, 1, 2))
train_labels_tensor = torch.Tensor(int_train_labels).type(torch.LongTensor)
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
train_loader = DataLoader(
    train_dataset,
    batch_size=2048,
    shuffle=True
)

test_images_tensor = torch.Tensor(normalized_test_images.transpose(0, 3, 1, 2))
test_labels_tensor = torch.Tensor(int_test_labels).type(torch.LongTensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(
    test_dataset,
    batch_size=2048,
    shuffle=False
)

In [8]:
for layer in vgg.classifier:
    if hasattr(layer, "weight"):
        nn.init.xavier_uniform_(layer.weight)
    if hasattr(layer, "bias"):
        nn.init.constant_(layer.bias.data, 0)

vgg.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(), lr=0.01, momentum=0.9)


num_epochs = 9
vgg.train()
for epoch in tqdm(range(num_epochs)):
    batch_loss = 0
    correct = 0
    num_batches = 0

    for batch_emb, batch_labels in train_loader:
        batch_emb = batch_emb.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()

        output = vgg(batch_emb)
        loss = criterion(output, batch_labels)

        loss.backward()
        optimizer.step()

        batch_loss += loss.item()
        _, pred = torch.max(output, dim=1)
        correct += torch.sum(pred == batch_labels).item()
        num_batches += batch_labels.size(0)

    avg_loss = batch_loss / len(train_loader)
    accuracy = 100 * correct / num_batches


    batch_loss = 0
    total_t = 0
    correct_t = 0
    with torch.no_grad():
        vgg.eval()

        for data_t, target_t in test_loader:
            data_t, target_t = data_t.to(device), target_t.to(device)

            outputs_t = vgg(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _, pred_t = torch.max(outputs_t, dim=1)

            correct_t = torch.sum(pred_t == target_t).item()
            total_t = target_t.size(0)

        avg_val_loss = batch_loss / total_t


    if (epoch + 1) % 3 == 0:
        print(f"Epoch {epoch+1}: Train Loss = {avg_loss:.4f}, Train Acc = {accuracy:.2f}%")
        print(f"validation loss: {avg_val_loss:.4f}, validation Acc: {(correct_t/total_t):.2f}%\n")


 20%|██        | 3/15 [01:37<06:30, 32.52s/it]

Epoch 3: Train Loss = 0.0221, Train Acc = 99.37%
validation loss: 0.0023, validation Acc: 0.85%



 40%|████      | 6/15 [03:14<04:51, 32.44s/it]

Epoch 6: Train Loss = 0.0074, Train Acc = 99.85%
validation loss: 0.0025, validation Acc: 0.86%



 60%|██████    | 9/15 [04:55<03:19, 33.25s/it]

Epoch 9: Train Loss = 0.0016, Train Acc = 99.98%
validation loss: 0.0027, validation Acc: 0.86%



 80%|████████  | 12/15 [06:36<01:40, 33.55s/it]

Epoch 12: Train Loss = 0.0007, Train Acc = 99.99%
validation loss: 0.0029, validation Acc: 0.86%



100%|██████████| 15/15 [08:19<00:00, 33.30s/it]

Epoch 15: Train Loss = 0.0004, Train Acc = 100.00%
validation loss: 0.0031, validation Acc: 0.86%



In [19]:
vgg.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for data_t, target_t in test_loader:
        data_t, target_t = data_t.to(device), target_t.to(device)
        
        outputs_t = vgg(data_t)
        _, pred_t = torch.max(outputs_t, dim=1)
        
        # CPUに移動してからリストに追加
        y_true.extend(target_t.cpu().numpy())
        y_pred.extend(pred_t.cpu().numpy())

# 分類レポート
print("=== Final Test Results ===")
print(classification_report(y_true, y_pred, 
                           target_names=[label.decode() for label in label_names]))

=== Final Test Results ===
              precision    recall  f1-score   support

    airplane       0.88      0.90      0.89      1000
  automobile       0.92      0.94      0.93      1000
        bird       0.81      0.82      0.82      1000
         cat       0.73      0.73      0.73      1000
        deer       0.83      0.84      0.84      1000
         dog       0.78      0.79      0.78      1000
        frog       0.91      0.90      0.90      1000
       horse       0.89      0.87      0.88      1000
        ship       0.93      0.93      0.93      1000
       truck       0.94      0.92      0.93      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

